# BFAST Impleentation

## Package installation 

In [ ]:
pkgTest <- function(x)
{
  if (x %in% rownames(installed.packages()) == FALSE) {
    install.packages(x, dependencies= TRUE)
  }
  library(x, character.only = TRUE)
}
neededPackages <- c("zoo", "bfast", "terra", "raster","dplyr")
for (package in neededPackages){pkgTest(package)}

## Create function to read data as time series

In [3]:
timeser <- function(val_array, time_array) {
    
    z <- zoo(val_array, time_array) # create zoo object
    
    year <- as.numeric(format(time(z), "%Y")) 
    Week <- as.numeric(format(time(z), "%W"))
    
    dt <- year + Week/53
    
    z <- aggregate(z, dt, mean)
    
    return(z)
}

## Perform Time Series Analysis to colombian cities

In [4]:
TimeSeries <- list.files('output/TS')
TimeSeries <- TimeSeries[1:(length(TimeSeries)-1)]

for (i in TimeSeries){
    ts <- read.csv(paste0('output/TS/', i))
    
    ts$Date <- as.Date(format(ts$Date, format = '%d'))
    group_by(ts, Date)
    
    tspx <- timeser(unlist(ts$value), as.Date(ts$Date, "%Y-%m-%d"))
    
    breaks <- bfastlite(tspx, response ~ trend, breaks = 5, order = 2)
    
    first_br <- time(tspx[breaks$breakpoints$breakpoints[1]])[1]
    
    
    png(paste0('output/',paste0(strsplit(i,'_')[[1]][1], strsplit(i,'_')[[1]][2]),'_trend.png'), height = 600, width = 1200)
    
    plot(breaks, main = paste0(strsplit(i,'_')[[1]][1], strsplit(i,'_')[[1]][2]), ylim = c(min(tspx), max(tspx)))
        
    start_col_lockdown <- 2020 + 03/12 + 25/ (30*12)
    end_col_lockdown <- 2020+ 06/12 + 30/(30*12)
    
    rect(start_col_lockdown,min(tspx), end_col_lockdown, max(tspx), col = '#FF00001A', ylim = c(min(tspx), max(tspx)))
    text((start_col_lockdown + end_col_lockdown)/2, max(tspx), 'LOCKDOWN', col = 'red')
    
    for (i in time(tspx[breaks$breakpoints$breakpoints])){
        yr <- i - i %% c(2018, 2019, 2020, 2021, 2022)[ i %% c(2018, 2019, 2020, 2021, 2022) < 1]
        month <- i %% c(2018, 2019, 2020, 2021, 2022)[ i %% c(2018, 2019, 2020, 2021, 2022) < 1]*12
        text(i + 0.1, y  = max(tspx), paste(as.character(yr), '-', as.character(round(month, 0))), ylim = c(min(tspx), 1.1*max(tspx)))
    }
    dev.off()
}

Warning message in zoo(val_array, time_array):
"some methods for "zoo" objects do not work if the index entries in 'order.by' are not unique"
